Book Recommender using KNN

Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


import os, sys
import re


Load dataset : books

In [2]:
df_books = pd.read_csv("D:/ML 3rd Year/Book Recommender/Books.csv/Books.csv")
df_books.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15432\2479420809.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("D:/ML 3rd Year/Book Recommender/Books.csv/Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


check column names

In [3]:
df_books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

Remove unwanted columns

In [4]:
# remove un wanted columns 
df_books = df_books[['ISBN', 'Book-Title', 'Book-Author']]
df_books.shape


(271360, 3)

Load dataset: ratings

In [5]:
df_ratings = pd.read_csv("D:/ML 3rd Year/Book Recommender/Ratings.csv/Ratings.csv")
df_ratings.head()
df_ratings.shape

(1149780, 3)

Checking for best model for Recommendation System

In [6]:
# Merge datasets
data = df_ratings.merge(df_books, on="ISBN", how="inner")

# Filter Data
min_book_ratings = 100
min_user_ratings = 200

book_counts = data["Book-Title"].value_counts()
user_counts = data["User-ID"].value_counts()

data = data[data["Book-Title"].isin(book_counts[book_counts >= min_book_ratings].index)]
data = data[data["User-ID"].isin(user_counts[user_counts >= min_user_ratings].index)]

data['Book-Title'] = data['Book-Title'].str.lower().str.strip()

# Ensure User-ID and ISBN are numeric
data['User-ID'] = pd.to_numeric(data['User-ID'], errors='coerce')
data['ISBN'] = LabelEncoder().fit_transform(data['ISBN'])  # Convert ISBN to numeric values
data = data.dropna(subset=['User-ID'])

# Define features and target
X = data[['User-ID', 'ISBN']]
y = data['Book-Rating']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
}

# Train and Evaluate Models
from sklearn.metrics import mean_absolute_error, r2_score

# Train and Evaluate Models
model_metrics = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store metrics
    model_metrics[model_name] = {
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2,
    }

    # Print metrics for each model
    print(f"{model_name} Performance Metrics:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R2 Score: {r2:.4f}")





Training KNN...
KNN Performance Metrics:
  MSE: 12.6124
  RMSE: 3.5514
  MAE: 2.4799
  R2 Score: 0.0036
Training DecisionTree...
DecisionTree Performance Metrics:
  MSE: 19.9688
  RMSE: 4.4686
  MAE: 2.4919
  R2 Score: -0.5776
Training LinearRegression...
LinearRegression Performance Metrics:
  MSE: 12.6204
  RMSE: 3.5525
  MAE: 2.9857
  R2 Score: 0.0029


Check Null Values

In [7]:
df_books.isnull().sum()

ISBN           0
Book-Title     0
Book-Author    2
dtype: int64

In [8]:
df_ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Drop Null Values

In [9]:
df_books.dropna(inplace=True)

In [10]:
df_books.isnull().sum()

ISBN           0
Book-Title     0
Book-Author    0
dtype: int64

Check shape of dataset

In [11]:
df_books.shape

(271358, 3)

In [12]:
df_ratings.shape

(1149780, 3)

Sort Ratings

In [13]:
# Calculate the count of ratings given by each user and store it in the 'ratings' Series
ratings = df_ratings['User-ID'].value_counts()
# Sort the 'ratings' Series in descending order based on the counts of user IDs
ratings.sort_values(ascending=False).head()

User-ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
Name: count, dtype: int64

check users with less than 200 ratings

In [14]:
len(ratings[ratings < 200])

104378

In [15]:
df_ratings['User-ID'].isin(ratings[ratings < 200].index).sum()

622224

In [16]:
df_ratings_rm = df_ratings[
  ~df_ratings['User-ID'].isin(ratings[ratings < 200].index)
]
df_ratings_rm.shape

(527556, 3)

Check books with less than 100 ratings

In [17]:
ratings = df_ratings['ISBN'].value_counts() 
ratings.sort_values(ascending=False).head()

ISBN
0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
Name: count, dtype: int64

In [18]:
len(ratings[ratings < 100])

339825

In [19]:
df_books['ISBN'].isin(ratings[ratings < 100].index).sum()

269422

In [20]:
df_ratings_rm = df_ratings_rm[
  ~df_ratings_rm['ISBN'].isin(ratings[ratings < 100].index)
]
df_ratings_rm.shape

(49781, 3)

Preprocess Dataset For Machine Learning

In [21]:
df_ratings_rm.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1469,277427,0060930535,0
1471,277427,0060934417,0
1474,277427,0061009059,9
1484,277427,0140067477,0


In [22]:
df_books.head()

,ISBN,Book-Title,Book-Author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [23]:
df = df_ratings_rm.pivot_table(index=['User-ID'],columns=['ISBN'],values='Book-Rating').fillna(0).T
df.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df.index = df.join(df_books.set_index('ISBN'))['Book-Title']

In [25]:
df = df.sort_index()
df.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df.loc["The Queen of the Damned (Vampire Chronicles (Paperback))"][:5]

User-ID
254     0.0
2276    0.0
2766    0.0
2977    0.0
3363    0.0
Name: The Queen of the Damned (Vampire Chronicles (Paperback)), dtype: float64

Create Model (KNN)

In [27]:
model = NearestNeighbors(metric='cosine')
model.fit(df.values)

NearestNeighbors(metric='cosine')

In [28]:
df.iloc[0].shape

(888,)

In [29]:
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
df.loc[title].shape

(888,)

In [30]:
distance, indice = model.kneighbors([df.loc[title].values], n_neighbors=6)

print(distance)
print(indice)

[[1.11022302e-16 5.17841186e-01 5.37633845e-01 7.34506886e-01
  7.44865700e-01 7.93983542e-01]]
[[612 660 648 272 667 110]]


In [31]:
df.iloc[indice[0]].index.values

array(['The Queen of the Damned (Vampire Chronicles (Paperback))',
       'The Vampire Lestat (Vampire Chronicles, Book II)',
       'The Tale of the Body Thief (Vampire Chronicles (Paperback))',
       'Interview with the Vampire',
       'The Witching Hour (Lives of the Mayfair Witches)', 'Catch 22'],
      dtype=object)

In [32]:
pd.DataFrame({
    'title'   : df.iloc[indice[0]].index.values,
    'distance': distance[0]
}) \
.sort_values(by='distance', ascending=True)

,title,distance
0,The Queen of the Damned (Vampire Chronicles (P...,1.110223e-16
1,"The Vampire Lestat (Vampire Chronicles, Book II)",5.178412e-01
2,The Tale of the Body Thief (Vampire Chronicles...,5.376338e-01
3,Interview with the Vampire,7.345069e-01
4,The Witching Hour (Lives of the Mayfair Witches),7.448657e-01
5,Catch 22,7.939835e-01


In [33]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
  try:
    book = df.loc[title]
  except KeyError as e:
    print('The given book', e, 'does not exist')
    return

  distance, indice = model.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : df.iloc[indice[0]].index.values,
      'distance': distance[0]
    }) \
    .sort_values(by='distance', ascending=True) \
    .head(6).values

  return [title, recommended_books]

check predictions

In [34]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

['The Queen of the Damned (Vampire Chronicles (Paperback))', array([['The Queen of the Damned (Vampire Chronicles (Paperback))',
        1.1102230246251565e-16],
       ['The Vampire Lestat (Vampire Chronicles, Book II)',
        0.5178411864186412],
       ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
        0.5376338446489461],
       ['Interview with the Vampire', 0.7345068863988313],
       ['The Witching Hour (Lives of the Mayfair Witches)',
        0.7448657003312193],
       ['Catch 22', 0.7939835419270879]], dtype=object)]


In [35]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["Where the Heart Is (Oprah's Book Club (Paperback))",
        2.220446049250313e-16],
       ['The Lovely Bones: A Novel', 0.7234864549790632],
       ['I Know This Much Is True', 0.7677075092617776],
       ['The Surgeon', 0.7699410973804288],
       ['The Weight of Water', 0.7708583572697412],
       ["I'll Be Seeing You", 0.8016210581447822]], dtype=object)]


In [36]:
books = get_recommends("The Surgeon")
print(books)

['The Surgeon', array([['The Surgeon', 0.0],
       ['The Honk and Holler Opening Soon', 0.7648533356656702],
       ['Last Man Standing', 0.7655158472957804],
       ['Manhattan Hunt Club', 0.7659204794397251],
       ['Seinlanguage', 0.7671219220376666],
       ["Where the Heart Is (Oprah's Book Club (Paperback))",
        0.7699410973804288]], dtype=object)]


In [37]:
books = get_recommends("The Honk and Holler Opening Soon")
print(books)

['The Honk and Holler Opening Soon', array([['The Honk and Holler Opening Soon', 3.3306690738754696e-16],
       ['The Woman Next Door', 0.7322366706282188],
       ["Big Cherry Holler: A Big Stone Gap Novel (Ballantine Reader's Circle)",
        0.7377929824449292],
       ['The Sigma Protocol', 0.7544535213933141],
       ['The Virgin Suicides', 0.7553329321750503],
       ['A Lesson Before Dying (Vintage Contemporaries (Paperback))',
        0.7564846145389318]], dtype=object)]
